<h2>1. Extracting information about Justin Trudeau using Guardian Media Group API </h1>

In [244]:
import os
import requests as re

api_key = 'test' # Replace it with your API key.
page = 1
results = []
more_pages = True

endpoint_url = "https://content.guardianapis.com/search?q=Justin Trudeau"

In [ ]:
import os
import requests as re

# Use the API key stored in an environment variable
api_key = os.environ["API_KEY"]

# Start on the first page of results
page = 1

# Initialize an empty list to store the results
results = []

# A flag to keep track of whether there are more pages of results
more_pages = True

# The base URL for the Guardian's Content API
endpoint_url = "https://content.guardianapis.com/search?q=Justin+Trudeau"

# While there are more pages of results
while more_pages:

    # Send a GET request to the API
    response = re.get(endpoint_url, params = { "api-key":api_key,
                "from-date": "2018-01-01",
                "to-date":"2022-12-30",
                "page":page,
                "page-size":50,
                })

    # If the request is successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Append the results to the list of all results
        results.extend(data["response"]["results"])

        # If there are more pages of results
        if data["response"]["pages"] > page:
            # Increment the page number
            page += 1
        else:
            # Set the flag to stop the loop
            more_pages = False
    else:
        # If the request is not successful, print the status code and raise an error
        print(f"Request failed with status code {response.status_code}")
        raise ValueError("Failed to retrieve results")


<h2>2. Count how many articles about Justin Trudeau have been posted since 01.01.2018 until
today </h2>

In [245]:
import pandas as pd

# # Uncomment the below line only when you want to update the csv file with new data
# # Create a Dataframe from the article results.
# df = pd.DataFrame(results)
#
# # Write to a csv file to avoid repeated API calls.
# df.to_csv("data.csv")

# Read data from csv file.
df = pd.read_csv("data.csv", parse_dates=["webPublicationDate"], index_col=["webPublicationDate"])

# Set the index to be the publication date, and convert it to a datetime object
df.index = pd.to_datetime(df.index.values)
df.index.name = "Date"

# Resamples the data by counting the ids of articles per day.
article_count_per_day = df.id.resample("D").count()

# The series created from resampling is converted to data frame and the values are renamed to number of articles.
data = article_count_per_day.to_frame("No of articles")

# Sum of articles published about Justin trudeau - 8194 articles in total
print("Since 01.01.2018 until 30.12.2022 there are {} articles published about Justin Trudeau".format(data["No of articles"].sum()))

# Print the data frame
data["No of articles"]


Since 01.01.2018 until 30.12.2022 there are 8194 articles published about Justin Trudeau


Date
2018-01-01    3
2018-01-02    3
2018-01-03    5
2018-01-04    1
2018-01-05    2
             ..
2022-12-26    2
2022-12-27    2
2022-12-28    1
2022-12-29    3
2022-12-30    4
Freq: D, Name: No of articles, Length: 1825, dtype: int64

<h2>3. Calculate the average of all days for the above-mentioned period from “No. of articles”.</h2>

In [246]:
# Summary statistics for No of articles to determine the nature of our data.
stats = data["No of articles"].describe()
stats

count    1825.000000
mean        4.489863
std         2.546719
min         0.000000
25%         3.000000
50%         4.000000
75%         6.000000
max        19.000000
Name: No of articles, dtype: float64

In [247]:
# Our data contains outliers. We will use median to calculate central tendancy / average
average = data["No of articles"].median()
print("Therefore, the average number of articles per day is {}".format(average))



Therefore, the average number of articles per day is 4.0


<h2> 4. In which section are most articles written?</h2>


In [248]:
# Group the dataframe by 'sectionName'
groups = df.groupby("sectionName")

# Count the number of articles per section and store in a variable
counts_per_section = groups.apply(lambda x : x["sectionName"].count())

# Convert the series to a dataframe and rename the column
counts_per_section = counts_per_section.to_frame("No of articles")

# Sort the dataframe by number of articles in descending order
counts_per_section = counts_per_section.sort_values(by="No of articles", ascending=False)

# Print the resulting dataframe
print(counts_per_section)


                                 No of articles
sectionName                                    
World news                                 1387
Sport                                      1300
Football                                    794
US news                                     654
Music                                       465
Australia news                              404
Opinion                                     389
Business                                    348
Politics                                    323
UK news                                     258
Film                                        239
Television & radio                          236
Environment                                 169
Media                                       152
Society                                     142
Culture                                     127
Stage                                       103
Technology                                  101
Fashion                                 

<h4> From the above  result it is evident that most articles about Justin Trudeau are written in <i> World News </i> section.</h4>

<h2>5. Show the evolution of the "No. of articles" over time for the above period.<h2>

In [249]:
import plotly.express as px

# Create a line chart using plotly express
fig = px.line(data,
              # Specify the x-axis to be the index of the dataframe
              x=data["No of articles"].index,
              # Specify the y-axis to be the values of the 'No of articles' column
              y=data["No of articles"].values,
              # Set the title of the chart
              title='Evolution of Daily Number of Articles about Justin Trudeau in The Guardian',
              # Rename the axis labels
              labels={'Date':'Timeline', 'y':'Number of Articles'})

# Show the chart
fig.show()


<h2>6. Are there any unusual events in the time series under investigation?
    If so, show these. Why are these unusual? (Define for yourself what you want to show by
        ordinary or unusual).
     Based on question one. Show the cause of the unusual event.</h2>

Justin Trudeau, prime minister of Canada and a renowned politician, his decisions affect millions of people.
Hence, it is natural that the number of articles published about him would depend on factors such as events or developments related to him.
So it is usual that number of articles about him vary from day to day. Also, the Guardian covers news from different part of the world. So
my thought is that it is usual if there are no articles written about Justin Trudeau on certain days. But if it continues for significant number
of days then something about it is unusual.
Upon investigation, it came to my attention that there are unusual events in the time series.
According to me, unusual event in the given time series is a sudden spike or drop in the number of articles specifically if it deviates significantly from underlying pattern or data trend.
We have one dimensional data, we will use a simple statistical method Inter Quartile Range(IQR) to find unusual events.


In [250]:
# Calculate the IQR by subtracting the 1st quartile from the 3rd quartile
iqr = data["No of articles"].quantile(0.75) - data["No of articles"].quantile(0.25)

# Calculate the lower and upper bounds for the outliers
lower_bound = data["No of articles"].quantile(0.25) - (3 * iqr)
upper_bound = data["No of articles"].quantile(0.75) + (3 * iqr)

# Find the data points that are outside the lower and upper bounds
anomalies = data[(data['No of articles'] < lower_bound) | (data['No of articles'] > upper_bound)]


In [251]:
# Check anomalies
anomalies

,No of articles
Date,
2018-06-11,17
2019-12-04,16
2020-03-13,19


In [252]:
import plotly.graph_objects as go

# Create a line chart with date on the x-axis and number of articles on the y-axis
fig = px.line(data, x=data.index, y=data["No of articles"].values, title='No of articles - UNSUPERVISED ANOMALY DETECTION', labels={'y':'No of articles'})

# Create a list of dates corresponding to the anomaly data points
anomaly_dates = anomalies.index

# Create a list of values corresponding to the anomaly data points
y_values = [anomalies.loc[i]['No of articles'] for i in anomaly_dates]

# Add a trace for the anomaly data points to the chart
fig.add_trace(go.Scatter(x=anomaly_dates, y=y_values, mode = 'markers',
                name = 'Anomaly',
                marker=dict(color='red',size=10)))

# Show the chart
fig.show()


As Justin Trudeau is a high-profile public figure, the news related to him is often significant and any events or developments concerning him can greatly influence the number of articles written about him. In our data, the following events are seen to have caused a significant spike in article counts:

1. On June 11, 2018, there was a significant increase in articles after US President Trump made negative statements about Justin Trudeau following the G7 summit.
2. On December 4, 2019, there was another spike in articles after a video of world leaders joking about Trump surfaced, which also included Justin Trudeau.
3. On March 13, 2020, major spike in articles was observed after it was announced that Justin Trudeau's wife had contracted COVID-19 and was going into self-isolation.



In [253]:
df.loc['2019-12-04']

,Unnamed: 0,id,type,sectionId,sectionName,webTitle,webUrl,apiUrl,fields,isHosted,pillarId,pillarName
Date,,,,,,,,,,,,
2019-12-04 19:55:51,1990,australia-news/2019/dec/05/morning-mail-trump-...,article,australia-news,Australia news,"Morning mail: Trump snubs Nato, Taylor inquiry...",https://www.theguardian.com/australia-news/201...,https://content.guardianapis.com/australia-new...,"{'headline': 'Morning mail: Trump snubs Nato, ...",False,pillar/news,News
2019-12-04 11:54:16,2180,us-news/2019/dec/04/footage-appears-to-show-wo...,article,us-news,US news,Footage appears to show world leaders joking a...,https://www.theguardian.com/us-news/2019/dec/0...,https://content.guardianapis.com/us-news/2019/...,{'headline': 'Footage appears to show world le...,False,pillar/news,News
2019-12-04 17:32:40,2197,us-news/2019/dec/04/trump-describes-trudeau-as...,article,us-news,US news,Trump cuts short Nato summit after fellow lead...,https://www.theguardian.com/us-news/2019/dec/0...,https://content.guardianapis.com/us-news/2019/...,"{'headline': ""Trump cuts short Nato summit aft...",False,pillar/news,News
2019-12-04 17:32:40,2220,us-news/2019/dec/04/trump-describes-trudeau-as...,article,us-news,US news,Trump cuts short Nato summit after fellow lead...,https://www.theguardian.com/us-news/2019/dec/0...,https://content.guardianapis.com/us-news/2019/...,"{'headline': ""Trump cuts short Nato summit aft...",False,pillar/news,News
2019-12-04 17:58:25,2231,politics/2019/dec/04/dont-mention-the-presiden...,article,politics,Politics,What's the joke? Mugged off Trump sulks his wa...,https://www.theguardian.com/politics/2019/dec/...,https://content.guardianapis.com/politics/2019...,"{'headline': ""What's the joke? Mugged off Trum...",False,pillar/news,News
2019-12-04 18:39:11,2235,politics/2019/dec/04/andrew-sparrows-election-...,article,politics,Politics,Andrew Sparrow's election briefing: Trump visi...,https://www.theguardian.com/politics/2019/dec/...,https://content.guardianapis.com/politics/2019...,"{'headline': ""Andrew Sparrow's election briefi...",False,pillar/news,News
2019-12-04 21:31:58,2487,politics/live/2019/dec/04/general-election-tor...,liveblog,politics,Politics,Boris Johnson denies joking about Donald Trump...,https://www.theguardian.com/politics/live/2019...,https://content.guardianapis.com/politics/live...,{'headline': 'Boris Johnson denies joking abou...,False,pillar/news,News
2019-12-04 18:16:45,2529,world/2019/dec/04/canada-logging-scars-forests...,article,world,World news,Long-term damage from logging hits ability of ...,https://www.theguardian.com/world/2019/dec/04/...,https://content.guardianapis.com/world/2019/de...,"{'headline': ""Long-term damage from logging hi...",False,pillar/news,News
2019-12-04 17:00:06,2784,world/2019/dec/04/how-does-nato-look-at-the-ag...,article,world,World news,How does Nato look at the age of 70? It's comp...,https://www.theguardian.com/world/2019/dec/04/...,https://content.guardianapis.com/world/2019/de...,"{'headline': ""How does Nato look at the age of...",False,pillar/news,News


<h2>9. Create a daily automated job that updates question 5 daily and creates an output that could be sent
to recipients who have not seen the data before. </h2>

Note : The solution code is saved in automated_task.py
Scheduler code is in scheduler.py